In [1]:
#先定义一个含有单层隐藏层的多层感知机
import torch
from torch import nn
from torch.nn import init
net = nn.Sequential(
    nn.Linear(4,3),
    nn.ReLU(),
    nn.Linear(3,1)
)
print(net)

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


In [2]:
X = torch.rand(2,4)
Y = net(X).sum()

访问模型参数

In [3]:
#使用Module类的parameters()或mnamed_parameters方法
#named_parameters方法以迭代器的形式返回，除了返回tensor，还会返回参数的名称
print(type(net.named_parameters()))
for name,param in net.named_parameters():
    print(name,param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [4]:
#d对于sequential的网络，可以用[]索引来访问俩民的任意一层的参数
for name,param in net[0].named_parameters():
    print(name,param.size(),type(param))
#返回的type是torch.nn.parameter.Paramete，是tensor的子类
#如果一个tensor是parameters，那么它会自动被添加到模型的参数列表里面

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [5]:
class MyModel(nn.Module):
    def __init__(self,**kwargs):
        super(MyModel,self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20,20))
        self.weight2 = torch.rand(20,20)
    def forward(self,x):
        pass
        

In [6]:
n = MyModel()
for name,param in n.named_parameters():
    print(name,param.size(),type(param))
#根据上面，weight1是Parameter类，那么会自动被添加到参数列表里，所以这里输出，只有weight1

weight1 torch.Size([20, 20]) <class 'torch.nn.parameter.Parameter'>


In [7]:
#Parameter是tensor，所以tensor的特性他都有
#这里用parameters来得到参数，所以parameters()方法是得到了list，可以用索引的到特定的参数
weight_0 = list(net[0].parameters())[0]
print(weight_0.size())


torch.Size([3, 4])


初始化模型参数

In [10]:
for name,param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param,mean=0,std=0.01)
        print(name ,param.data)
#正态分布初始化

0.weight tensor([[ 0.0189,  0.0113,  0.0122,  0.0175],
        [ 0.0170,  0.0082, -0.0108, -0.0151],
        [-0.0031, -0.0096,  0.0076,  0.0052]])
2.weight tensor([[ 0.0107, -0.0045,  0.0040]])


In [12]:
#常数初始化参数
for name,param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param,val = 0)
        print(name,param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


自定义初始化方法

In [13]:
#有些方法没有在init模块中给出，就需要实现一个初始化方法
#torch.nn.init.normal_的实现
def normal_(tensor,mean=0,std=1):
    with torch.no_grad():
        return tensor.normal_(mean,std)
#本质是一个inpalce改变tensor值的函数，不计梯度

In [16]:
#实现一个自定义的初始化方法
#这里让权重有一半的概率初始化为0，另一半概率初始化为-10，-5和5，10两个区间里均匀分布的随机数
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10,10)#对tensor进行原地修改，初始化为区间-10到10内的随机数
        tensor *=(tensor.abs()>=5).float()#绝对值小于5的数被初始化为0
for name,param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name,param.data)

0.weight tensor([[ 0.0000,  8.6678,  9.9834, -8.5890],
        [-8.2359, -6.9382,  9.0086,  0.0000],
        [-7.4985,  9.3658, -0.0000,  8.1228]])
2.weight tensor([[0.0000, 5.3146, 0.0000]])


In [17]:
#还可以通过直接改变参数data的值（因为改data不影响梯度
for name,param in net.named_parameters():
    if 'bias' in name:
        param.data+=1
        print(name,param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


共享模型参数

In [36]:
#如果传入sequtential的模块是同一个module实例的话，那么参数也是共享的
linear = nn.Linear(1,1,bias = False)
net = nn.Sequential(linear,linear)
print(net)
for name,param in net.named_parameters():
    init.constant_(param,3)
    print(name,param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [37]:
#在内存中这两个先行曾是一个对象
print(id(net[0])==id(net[1]))

True


In [38]:
#模型参数种包含梯度，所以在反向传播计算时，哲别共享的参数的梯度也是累加的
x = torch.ones(1,1)
y = net(x).sum()
print(y)

tensor(9., grad_fn=<SumBackward0>)


In [39]:

y.backward()
print(net[0].weight.grad)#单次梯度时3，两次就是6

tensor([[6.]])
